# Multi-agent Examples

In this example I show an example of an agent that writes a story using a series of steps.

1. The user provides an idea for their story.
2. The `Outline Bot` generates an outline for the story including section titles and descriptions.
3. The first section content is created by the `Story Bot` from section title/description and the overall story description.
4. The `Summary Bot` creates a summary of the newly generated chapter.
5. The section summary is passed to the `Story Bot` along with section title/description to generate the next section.
6. All sections follow steps 3-5.
...
7. The sections are combined into a full story.

![Story bot design diagram](https://storage.googleapis.com/public_data_09324832787/story_bot_design.svg)

In [1]:
import sys
sys.path.append('..')

import simplechatbot
from simplechatbot.openai import OpenAIChatBot

In [2]:
keychain = simplechatbot.APIKeyChain.from_json_file('../keys.json')

def new_chatbot(system_prompt: str):
    return OpenAIChatBot.new(
        model_name = 'gpt-4o-mini', 
        api_key=keychain['openai'],
        system_prompt=system_prompt,
    )

def stream_it(chatbot: simplechatbot.ChatBot) -> simplechatbot.ChatResult:
    stream = chatbot.chat_stream(None, add_to_history=False)
    for chunk in stream:
        print(chunk.content, end='', flush=True)
    return stream.result()

In [3]:
import pydantic

from typing import Optional

#from pydantic import BaseModel, Field
import pydantic

class StoryOutline(pydantic.BaseModel):
    """Outline of the story."""

    story_topic: str = pydantic.Field(description="The topic of the story.")

    part1_title: str = pydantic.Field(description="Title of Part 1 of the story.")
    part1_description: str = pydantic.Field(description="Longer description of part 1.")

    part2_title: str = pydantic.Field(description="Title of Part 2 of the story.")
    part2_description: str = pydantic.Field(description="Longer description of part 2.")

    part3_title: str = pydantic.Field(description="Title of Part 3 of the story.")
    part3_description: str = pydantic.Field(description="Longer description of part 3.")


system_prompt = '''
The user will provide you a description of a story, and you must create a chapter outline with titles and brief descriptions.
Each section should contain a title and a brief description of what happens in that section.
The sections should all be part of a single narrative ark, but each section should have a complete beginning, middle, and end.
'''
outline_bot = new_chatbot(system_prompt)

q = f'Write an outline for a story about two friends who met when they were young and then lost touch. They meet again as adults and have to navigate their new relationship.'
outline: StoryOutline = outline_bot._model.with_structured_output(StoryOutline).invoke(q)
#chatbot_joke = chatbot_base.clone(model_factory=lambda llm: llm.with_structured_output(Joke), clear_tools=True)
#chatbot_joke.invoke('Tell me a joke about a chicken', max_tokens=50)

#chatbot_joke.invoke('Tell me a joke about a chicken', max_tokens=50)
outline

StoryOutline(story_topic='Rekindling Old Friendships', part1_title='The Childhood Bond', part1_description='Two young boys, Alex and Ben, meet in their neighborhood and quickly become inseparable friends. They share adventures, secrets, and dreams, creating a strong bond that seems unbreakable. However, due to family circumstances, Ben moves away, and they lose touch. Their childhood memories linger, but life pulls them in different directions.', part2_title='Years Apart', part2_description='As the years pass, Alex and Ben grow up in different environments, shaping their identities and interests. Alex becomes a successful artist in his hometown, while Ben moves to a big city to pursue a career in technology. They both think of each other fondly but never make the effort to reconnect. Their lives seem fulfilling, but a sense of something missing remains.', part3_title='A Chance Encounter', part3_description="Fate intervenes when Alex visits the city for an art exhibition and unexpectedl

In [4]:
def first_section_writer_prompt(story_topic: str, section_title: str, section_description: str):

    return f'''
    You are to write the first section of a larger story based on a general story topic, the name of the section, and a longer description of what happens.

    Your responses should only include text that is part of the story.

    General story topic: "{story_topic}"

    Section title: "{section_title}"

    Description: "{section_description}"

    '''
cb1 = new_chatbot(first_section_writer_prompt(outline.story_topic, outline.part1_title, outline.part1_description))
ch1_result = stream_it(cb1)

**The Childhood Bond**

The sun hung low in the sky, casting golden rays across the quiet suburban street, where laughter and the sounds of summer filled the air. Alex sat on the curb, swinging his feet back and forth, watching the world around him with wide, curious eyes. It was a typical afternoon, the kind that promised adventure to any child willing to seek it. As he tossed a small pebble into the gutter, the sound of approaching footsteps caught his attention.

"Hey!" a voice called out, bright and full of energy. Alex turned to see a boy his age, with wild brown hair and a grin that seemed to stretch from ear to ear. "Wanna play?"

"Sure!" Alex replied, jumping up from his spot. "What do you want to do?"

"Let’s go explore the woods!" Ben suggested, pointing toward the dense line of trees at the end of the street. The woods had always been a place of mystery—filled with whispered tales of hidden treasures and secret paths.

They bounded down the sidewalk, their laughter echoing i

In [5]:
def summary_prompt(story_so_far: str) -> str:
    return f'''
    You need to create a summary of the following section of a story that can be used as a starting point for writing the next section.

    Your responses should only include text that is part of the story.

    The previous section of the story:

    {story_so_far}
    '''
cb1_sum = new_chatbot(summary_prompt(ch1_result.content))
ch1_sum = stream_it(cb1_sum)

**Summary: The Childhood Bond**

On a sunny afternoon, Alex meets Ben, a boy with wild hair, and they embark on a series of adventures in the nearby woods, building a strong friendship filled with laughter and imaginative play. As summer ends, they face the heartbreaking news that Ben's family is moving away, leading to emotional goodbyes and promises to remain best friends. They create lasting memories, including carving their initials into a tree and burying a time capsule. After Ben leaves, Alex struggles with the absence of his friend but holds onto the cherished memories of their childhood bond, visiting their secret spot and reflecting on what they shared.

In [6]:
def next_section_writer_prompt(story_topic: str, section_title: str, section_description: str, story_so_far: str):
    return f'''
    You are to write the first section of a larger story based on a general story topic, the name of the section, and a longer description of what happens.

    Your responses should only include text that is part of the story.

    General story topic: "{story_topic}"

    Summary of previous section: "{story_so_far}"

    Section title: "{section_title}"

    Description: "{section_description}"

    '''
cb2 = new_chatbot(next_section_writer_prompt(outline.story_topic, outline.part2_title, outline.part2_description, ch1_sum.content))
ch2_result = stream_it(cb2)

**Years Apart**

As the sun dipped below the horizon, painting the sky in hues of orange and purple, Alex stood in his small studio, surrounded by canvases that captured the beauty of his hometown. He dipped his brush into a palette of colors, each stroke a reflection of the life he had built since Ben moved away. The quaint streets, the familiar faces, and the gentle rustling of leaves in the nearby woods were his muses, but amidst the vibrant scenes he created, there lingered an emptiness. He often found himself staring at the tree they had carved their initials into, a bittersweet reminder of a friendship that had weathered the storms of childhood but faded with time.

Meanwhile, in the bustling heart of the city, Ben was immersed in the world of technology, surrounded by the hum of computers and the glow of screens. He had traded the rustling leaves for the cacophony of urban life, and although he thrived in his career, he often found himself reminiscing about the adventures he and

In [7]:
cb2_sum = new_chatbot(summary_prompt(ch2_result.content))
ch2_sum = stream_it(cb2_sum)

As Alex prepares for his exhibition, he feels a strong desire to reconnect with Ben, the memories of their childhood friendship weighing heavily on his heart. Meanwhile, Ben reflects on their shared past while going through an old notebook filled with dreams they once had, igniting a yearning to reach out to Alex. Both men stand at a pivotal moment in their lives, ready to bridge the gap that years of distance have created, unaware that fate is orchestrating their reunion.

In [8]:
cb3 = new_chatbot(next_section_writer_prompt(outline.story_topic, outline.part2_title, outline.part2_description, ch1_sum.content))
ch3_result = stream_it(cb3)

**Years Apart**

The years drifted by like leaves caught in a gentle autumn breeze, swirling through the seasons of life. Alex found solace in the vibrant colors of his paintbrush, each stroke on the canvas a tribute to the adventures he once shared with Ben. The small town felt comforting, its familiar streets and friendly faces wrapping around him like a warm blanket. Yet, in the quiet moments, when the sun dipped below the horizon and shadows danced across his studio, a flicker of nostalgia would ignite in his heart, reminding him of laughter echoing through the woods and the promise of forever friendship whispered beneath the tall oak tree.

Meanwhile, in the bustling heart of the city, Ben navigated the concrete labyrinth of his new life, his wild hair now tamed into a professional style that suited his ambitions in technology. He thrived amidst the buzz of innovation and the thrill of new challenges. Yet, even as he climbed the corporate ladder, the faint memory of a childhood fr

In [9]:
cb3_sum = new_chatbot(summary_prompt(ch3_result.content))
ch3_sum = stream_it(cb3_sum)

As Alex waited for a reply, memories of their shared past flooded his mind. He recalled the laughter, the adventures, and the promises made under the oak tree. Each moment felt like a thread pulling him back to a time when their friendship was unbreakable. Meanwhile, Ben sat in his modern apartment, staring at the email notification that had just pinged on his screen. A rush of emotions surged through him as he recognized Alex’s name. He hesitated for a moment, grappling with the weight of the years that had passed and the life he had built without his childhood friend. 

Finally, with a deep breath, he opened the email. The warmth of Alex's words brought a smile to his face, and he felt a longing to reconnect. As he began to type his response, he considered how much he missed the simplicity of their friendship and the joy that came from just being together. The excitement of rediscovering their bond filled him with a sense of purpose. 

Both men stood on the precipice of change, their

In [10]:
story = dict()
story[outline.part1_title] = ch1_result.content
story[outline.part2_title] = ch2_result.content
story[outline.part3_title] = ch3_result.content
story.keys()

dict_keys(['The Childhood Bond', 'Years Apart', 'A Chance Encounter'])

In [11]:
for title, content in story.items():
    print(f'\n\n\n{title}\n\n{content}\n\n\n')




The Childhood Bond

**The Childhood Bond**

The sun hung low in the sky, casting golden rays across the quiet suburban street, where laughter and the sounds of summer filled the air. Alex sat on the curb, swinging his feet back and forth, watching the world around him with wide, curious eyes. It was a typical afternoon, the kind that promised adventure to any child willing to seek it. As he tossed a small pebble into the gutter, the sound of approaching footsteps caught his attention.

"Hey!" a voice called out, bright and full of energy. Alex turned to see a boy his age, with wild brown hair and a grin that seemed to stretch from ear to ear. "Wanna play?"

"Sure!" Alex replied, jumping up from his spot. "What do you want to do?"

"Let’s go explore the woods!" Ben suggested, pointing toward the dense line of trees at the end of the street. The woods had always been a place of mystery—filled with whispered tales of hidden treasures and secret paths.

They bounded down the sidewalk, t